# Diffusion equation in the case where I know the analytical solution.

Let's construct a test problem, to determine if the calculations are correct

Now I want to solve the diffusion equation in the case where I know the analytical solution.

and this is linear in time (but quadratic in space), that is, I'm choosing the following analytical solution:

u= 1 + x^2 + alpha *y^2 + beta * t

Thus, we will have:

f(x,y)= beta - 2 - 2*alpha

Diricelet BC: u_D= 1 + x^2 + alpha *y^2 + beta * t 

Initial condition: u0= 1 + x^2 + alpha *y^2 +


In [1]:
from petsc4py import PETSc
from mpi4py import MPI
import ufl
from dolfinx import mesh, fem
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
import numpy
t = 0  # Start time
T = 2  # End time
num_steps = 20  # Number of time steps
dt = (T - t) / num_steps  # Time step size
alpha = 3
beta = 1.2

In [2]:
# Now I define domain, mesh and thus vector space 

nx, ny = 5, 5
domain = mesh.create_unit_square(MPI.COMM_WORLD, nx, ny, mesh.CellType.triangle)
V = fem.functionspace(domain, ("Lagrange", 1))

Now I define the exact solution:

In [3]:
class exact_solution():
    def __init__(self, alpha, beta, t):
        self.alpha = alpha
        self.beta = beta
        self.t = t

    def __call__(self, x):  #la classe __call__ mi permette di chiamare gli oggetti della classe come se fossero delle funzioni 
        return 1 + x[0]**2 + self.alpha * x[1]**2 + self.beta * self.t
    
u_exact= exact_solution(alpha,beta,t)

Now I need to define the BC (and thus also the dofs that belong to the boundary where to apply it).

In [4]:
# define the BC
uD=fem.Function(V)
uD.interpolate(u_exact)

# now I identify the facets on the boundary and the dofs on the boundary 
tdim=domain.topology.dim
fdim=tdim-1
domain.topology.create_connectivity(fdim,tdim) # create a mapping between facets and mesh cells 
boundary_facets=mesh.exterior_facet_indices(domain.topology) # identify the facets on the boundary 

# now I need to identify the dofs of the facets on the boundary 
boundary_dofs=fem.locate_dofs_topological(V,fdim,boundary_facets)

# so I define the BCs (Dirichlet)
bc=fem.dirichletbc(uD,boundary_dofs)

In [5]:
# now I define u_n at the initial step n=0 (so t=0)

u_n=fem.Function(V)
u_n.interpolate(u_exact) # since t is still equal to 0 it's as if I'm calculating the initial solution

# so I define f 
f=fem.Constant(domain,beta-2-2*alpha)

Now that I have defined the variables I can formulate the variational problem.

In [6]:
u=ufl.TrialFunction(V)
v=ufl.TestFunction(V)

F=u*v*ufl.dx+dt*ufl.inner(ufl.grad(u),ufl.grad(v))*ufl.dx-(u_n+dt*f)*v*ufl.dx

a=fem.form(ufl.lhs(F))
L=fem.form(ufl.rhs(F))

Now I create matrix A and vector b of the variational problem expressed as a linear system.

Remember that A will be constant as t-steps vary,

while b will vary as it will contain information about u at the previous time-step.

In [7]:
A = assemble_matrix(a, bcs=[bc])
A.assemble()
b = create_vector(L)
uh = fem.Function(V)

In [8]:
from petsc4py import PETSc

solver=PETSc.KSP().create(domain.comm)

solver.setOperators(A)

solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

In [9]:
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc

for n in range(num_steps):
    # Update Dirichlet boundary condition
    u_exact.t += dt # update the value of t in the exact solution (which will be used to define the Dirichlet BC)
    uD.interpolate(u_exact)

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0) # so I reset the value of b (because I will have to update it with the information
# contained by u_n at the previous time-step)
    assemble_vector(b, L)

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [a], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc])

    # Solve linear problem
    solver.solve(b, uh.x.petsc_vec)
    uh.x.scatter_forward()

    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh.x.array

Now I check the accuracy of the problem solution.

In [10]:
# Compute L2 error and error at nodes
V_ex = fem.functionspace(domain, ("Lagrange", 2)) # define the vector space of quadratic functions on the single element 
u_ex = fem.Function(V_ex) 
u_ex.interpolate(u_exact)

error_L2 = numpy.sqrt(domain.comm.allreduce(fem.assemble_scalar(fem.form((uh - u_ex)**2 * ufl.dx)), op=MPI.SUM))
if domain.comm.rank == 0:
    print(f"L2-error: {error_L2:.2e}")

# Compute values at mesh vertices
error_max = domain.comm.allreduce(numpy.max(numpy.abs(uh.x.array - uD.x.array)), op=MPI.MAX)
if domain.comm.rank == 0:
    print(f"Error_max: {error_max:.2e}")

L2-error: 2.83e-02
Error_max: 1.78e-15


As expected, the error at the dofs is practically machine error!!!!